In [1]:
using DataFrames, CSV
using LinearAlgebra
using Plots
using Gurobi, JuMP

In [2]:
F = DataFrame(CSV.File("/Users/thiagodesouza/Codes/6.7201/Pset_2/facilities.csv"))

,x,y,supply
,Float64,Float64,Float64
1,0.0285239,1.71448,86.7759
2,0.508327,2.76107,140.987
3,3.60514,4.41322,213.104
4,8.43778,1.25384,102.001
5,3.76959,5.45553,409.883
6,4.89495,8.96613,576.407
7,1.80321,6.49412,205.929
8,3.25198,0.451999,325.911
9,4.39766,4.72416,317.815


In [3]:
C = DataFrame(CSV.File("/Users/thiagodesouza/Codes/6.7201/Pset_2/customers.csv"))

,x,y,demand
,Float64,Float64,Float64
1,7.37441,2.06318,102.872
2,1.00208,7.29743,6.83076
3,3.61792,1.25644,138.975
4,2.27311,2.9264,26.4615
5,0.765783,9.5186,97.0179
6,2.03323,9.76306,35.1446
7,9.92024,1.5894,152.489
8,4.3794,9.87982,69.411
9,5.69553,8.78813,114.95


In [4]:
d = C[:, 3]

500-element Vector{Float64}:
 102.87162507351552
   6.8307637892982145
 138.97458746312324
  26.461541484160307
  97.01794464124987
  35.14459189894622
 152.48859238864728
  69.41099081174202
 114.94952587053982
  87.35517707844693
   4.200405200412108
  27.38748564962148
  10.217898791964329
   ⋮
   8.344162861299406
  52.70573158819014
  14.780295609922753
  20.794778664732696
  32.24342069283391
  12.741665178545905
  20.133412899784005
   9.65271384073412
  41.88658649771853
  34.543666814888084
  60.055128143355425
 116.1817044511752

In [5]:
s = F[:, 3]

100-element Vector{Float64}:
  86.77590953445319
 140.9867222243375
 213.10358856417525
 102.00143123380172
 409.8834071348691
 576.4065169876949
 205.92873048401964
 325.9114536601554
 317.815075142178
 201.91942553359718
 178.1760943163833
 123.01794599797941
 273.513770006315
   ⋮
 384.20150514224207
 233.27188418870574
 822.3629551502889
 151.41857256148128
 280.07469460750195
 222.250100838334
 376.471111551249
 350.74138862012927
 202.89264599147853
 443.21993627174766
 198.22717226653944
 273.7267560764144

In [6]:
c = zeros(100,500)

for i = 1:100
    
    for j = 1:500
        
        c[i,j] = (((F[i,1] - C[j,1])^2 + (F[i,2] - C[j,2])^2)^0.5)
        
    end
end

c
        
        
        

100×500 Matrix{Float64}:
 7.35416   5.6672   3.6185    2.55087  …  2.95463  8.94706   2.59445
 6.90146   4.56316  3.45448   1.77251     2.72912  8.51112   1.48807
 4.44186   3.88518  3.15681   1.99623     2.87529  5.93534   3.0164
 1.33633   9.582    4.81987   6.38754     5.46799  0.826434  8.39598
 4.95003   3.32442  4.20184   2.9387      3.929    6.31277   3.3985
 7.33475   4.23545  7.81475   6.58425  …  7.60444  8.17397   6.38567
 7.1184    1.13452  5.54315   3.59853     5.02762  8.52558   2.55238
 4.4261    7.2057   0.883758  2.66099     1.18579  5.89817   4.63337
 3.99273   4.26049  3.55431   2.7831      3.42441  5.38672   3.83416
 5.64988   6.65181  1.81356   2.25399     1.41513  7.18103   3.77149
 7.35331   6.64739  8.95315   8.21062  …  8.92567  7.71659   8.46992
 4.88659   7.20051  1.23382   2.65386     1.29154  6.36051   4.50169
 6.03379   5.72909  2.28088   1.62719     1.62136  7.62      2.77518
 ⋮                                     ⋱                     
 1.77129   6.99895

In [43]:
model = Model(Gurobi.Optimizer)

@variable(model, x[1:100, 1:500] >= 0)

# d is the demand vector
@constraint(model, [j = 1:500], sum(x[i,j] for i=1:100) >= d[j]) 

# s is the supply vector
@constraint(model, [i = 1:100], sum(x[i,j] for j=1:500) <= s[i])

# c is the Euclidean distance matrix (100 x 500)
@objective(model, Min, sum(c[i,j]*x[i,j] for i =1:100, j=1:500))

optimize!(model)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-03
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 600 rows, 50000 columns and 100000 nonzeros
Model fingerprint: 0x34f7f735
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-02, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-01, 1e+03]

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve time: 0.04s
Presolved: 600 rows, 50000 columns, 100000 nonzeros

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 5.000e+04
 Factor NZ  : 6.521e+04 (roughly 20 MB of memory)
 Factor Ops : 7.808e+06 (less than 1 second per iteration)
 Threads    : 4

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.36188104e+07  0.0

In [44]:
x_opt = value.(x)

100×500 Matrix{Float64}:
  0.0     0.0  0.0  0.0  0.0   0.0     …  0.0  0.0   0.0     0.0  0.0  0.0
  0.0     0.0  0.0  0.0  0.0   0.0        0.0  0.0   0.0     0.0  0.0  0.0
  0.0     0.0  0.0  0.0  0.0   0.0        0.0  0.0   0.0     0.0  0.0  0.0
  0.0     0.0  0.0  0.0  0.0   0.0        0.0  0.0   0.0     0.0  0.0  0.0
  0.0     0.0  0.0  0.0  0.0   0.0        0.0  0.0   0.0     0.0  0.0  0.0
  0.0     0.0  0.0  0.0  0.0  35.1446  …  0.0  0.0   0.0     0.0  0.0  0.0
  0.0     0.0  0.0  0.0  0.0   0.0        0.0  0.0   0.0     0.0  0.0  0.0
  0.0     0.0  0.0  0.0  0.0   0.0        0.0  0.0   0.0     0.0  0.0  0.0
  0.0     0.0  0.0  0.0  0.0   0.0        0.0  0.0   0.0     0.0  0.0  0.0
  0.0     0.0  0.0  0.0  0.0   0.0        0.0  0.0   0.0     0.0  0.0  0.0
  0.0     0.0  0.0  0.0  0.0   0.0     …  0.0  0.0   0.0     0.0  0.0  0.0
  0.0     0.0  0.0  0.0  0.0   0.0        0.0  0.0   0.0     0.0  0.0  0.0
  0.0     0.0  0.0  0.0  0.0   0.0        0.0  0.0   0.0     0.0  0.0  0.0


In [53]:
closest_fac = zeros(500,1)

for j = 1:500
    
    closest_fac[j] = argmin(c[:, j])
    
end

closest_fac

500×1 Matrix{Float64}:
 87.0
 73.0
 25.0
 37.0
 99.0
 99.0
 33.0
 19.0
 85.0
  1.0
 65.0
  2.0
  9.0
  ⋮
 75.0
 67.0
 28.0
 97.0
 67.0
 80.0
 71.0
 39.0
 11.0
 77.0
  4.0
 34.0

In [57]:
count = 0

for j = 1:500
   
    # closest_fac is a vector (500 x 1) of the closest facility to each customer
    if x_opt[trunc(Int, closest_fac[j]),j] == sum(x_opt[i, j] for i=1:100)
        count +=1
    end
    
end

count

220